In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

import utils.read_data as rd
import utils.preprocessing as pre

In [ ]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/'

In [ ]:
df_horse = rd.read_horse_csv(DATA_PATH)

In [ ]:
df_race = rd.read_race_csv(DATA_PATH)

In [ ]:
df_horse = df_horse.sort_values("race_id")

In [ ]:
len(df_horse)

In [ ]:
df_horse = df_horse.iloc[int(29 * len(df_horse)/30):-1, :].reset_index(drop=True)
# df_horse = df_horse.iloc[:100, :].reset_index(drop=True)
#df_horse_half

In [ ]:
horse_ids = list(set(df_horse['horse_id']))
horse_ids

In [ ]:

df = pd.merge(df_horse, df_race, on='race_id', how='left')
arr_df = []
for horse_id in tqdm(horse_ids):
    df_horse_tmp = df.query('horse_id == @horse_id')
    df_horse_tmp = df_horse_tmp.sort_values("race_id")
    arr_df.append(df_horse_tmp)

In [ ]:
arr_res = Parallel(n_jobs=-1, verbose=8)([delayed(pre.add_n_race_data_para)(df) for df in arr_df])

In [ ]:
result = pd.concat((df.T for df in tqdm(arr_res)), axis=1).T

#result = pd.DataFrame()
#for df in tqdm(arr_res):
#    result = pd.concat([result, df])

In [ ]:
result

In [ ]:
result.reset_index(inplace=True, drop=True)

In [ ]:
result.to_csv(GOOGLE_DRIVE_PATH + '/data/small_horse_race.csv')

# テスト

In [ ]:
len(df_horse)

In [ ]:
df_horse.groupby('horse_id')['race_id'].count().max()

In [ ]:
df_horse_min = df_horse.iloc[0:10000:10, :]

In [ ]:
df_horse_min

In [ ]:
def add_n_race_data(df_race, df_horse, n_race=1):
    res = pd.DataFrame()
    horse_ids = list(set(df_horse['horse_id']))
    df = pd.merge(df_horse, df_race, on='race_id', how='left')
    for horse_id in tqdm(horse_ids):
        df_horse_tmp = df.query('horse_id == @horse_id')
        df_horse_tmp = df_horse_tmp.sort_values("race_id")
        race_ids = list(set(df_horse_tmp['race_id']))
        for race_id in race_ids:
            df_race_tmp = df_horse_tmp.query('race_id == @race_id').reset_index(drop=True)
            df_horse_low_race_id = df_horse_tmp.query('race_id <= @race_id')
            columns = df_horse_low_race_id.columns
            for n in range(1, n_race+1):
                add_columns = list(map(lambda c: c+f"_{n}", columns))
                add_race = df_horse_low_race_id.iloc[-1-n:-n, :].reset_index(drop=True)
                add_race.columns = add_columns
                df_race_tmp = pd.concat([df_race_tmp, add_race], axis=1)
                    
            res = pd.concat([res, df_race_tmp])

    return res

In [ ]:
result = add_n_race_data(df_race, df_horse_min, n_race=15)
#result = add_n_race_data(df_race, df_horse_min, n_race=5)

In [ ]:
result

In [ ]:
result.to_csv(GOOGLE_DRIVE_PATH + '/data/horse_race.csv')

In [ ]:
result.info()

In [ ]:
for column in result.columns:
    print(column)

In [ ]:
def add_n_race_data_para(df):
    res = pd.DataFrame()
    n_race = 15
    df = df.sort_values("race_id")
    race_ids = list(set(df['race_id']))
    for race_id in race_ids:
        df_race_tmp = df.query('race_id == @race_id').reset_index(drop=True)
        df_horse_low_race_id = df.query('race_id <= @race_id')
        columns = df_horse_low_race_id.columns
        for n in range(1, n_race+1):
            add_columns = list(map(lambda c: c+f"_{n}", columns))
            add_race = df_horse_low_race_id.iloc[-1-n:-n, :].reset_index(drop=True)
            add_race.columns = add_columns
            df_race_tmp = pd.concat([df_race_tmp, add_race], axis=1)

        res = pd.concat([res, df_race_tmp])

    return res

In [ ]:
def create_horse_data(n_race, df_race, df_horse):
    df_race = df_race.reset_index(drop=True)
    race_id = df_race.iloc[0, 0]
    df_horse.sort_values("race_id", inplace=True)
    df_hourse = df_horse.query('race_id < @race_id')
    columns = df_horse.columns
    for n in range(1, n_race+1):
        add_columns = list(map(lambda c: c+f"_{n}", columns))
        add_race = df_horse.iloc[-1-n:-n, :].reset_index(drop=True)
        add_race.columns = add_columns
        df_race = pd.concat([df_race, add_race], axis=1)
    return df_race

In [ ]:
def extract_previous_race(n, df):
    return df.loc[:, f"race_year_{n}": f"horse_ranking_index_{n}"]

In [ ]:
df_horse_test = df_horse.query('horse_id == 2010103480')
df_horse_test

In [ ]:
# 3レース前までのデータを列に追加
result = create_horse_data(3, df_race_test, df_horse_test)
result

In [ ]:
extract_previous_race(1, result)

# 古いコード

In [ ]:
df = pd.read_csv("data/data_2400.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df["horse_name"].value_counts()

## 指定した馬のnレース前までのデータを列に追加する

In [ ]:
test_df = df[df["horse_name"] == "ステイブラビッシモ"].loc[:, :"horse_ranking_index"]
test_df

## 日時情報を整形
日時情報を使ってデータフレームをソートしたデータから、nレース前までのデータを取得するために行う。

In [ ]:
def add_date_data(df, year_column_name, month_column_name,  day_column_name):
    df["date"] = df.apply(lambda row: f"{row[year_column_name]}-{row[month_column_name]}-{row[day_column_name]}", axis=1)
    df["date"] = pd.to_datetime(df["date"])

In [ ]:
add_date_data(test_df, "race_year", "race_month", "race_day")
test_df.info()

## nレース前までのデータを列に追加する

In [ ]:
def create_horse_data(n_race, df):
    df.sort_values("date", inplace=True)
    race_df = df.iloc[-1:, :].reset_index(drop=True)
    columns = race_df.columns
    for n in range(1, n_race+1):
        add_columns = list(map(lambda c: c+f"_{n}", columns))
        add_race = df.iloc[-1-n:-n, :].reset_index(drop=True)
        add_race.columns = add_columns
        race_df = pd.concat([race_df, add_race], axis=1)
    return race_df

In [ ]:
# 3レース前までのデータを列に追加
result = create_horse_data(3, test_df)
result

In [ ]:
def extract_previous_race(n, df):
    return df.loc[:, f"race_year_{n}": f"horse_ranking_index_{n}"]

In [ ]:
extract_previous_race(1, result)

In [ ]:
extract_previous_race(2, result)

In [ ]:
extract_previous_race(3, result)